##  Create S3 bucket using Boto3

Boto 3 is the AWS Software Development Kit(SDK) for Python, which allows us to directly create, update and delete AWS resources from our Python scripts.

Boto3 makes it easy to integrater our Python application library or script with AWS services including Amazon S3, Amazon EC2, Amazon DynamoDB and more.

In [9]:
import boto3
s3 = boto3.resource('s3')
bucket_name = 'stockpricepredictionxgb'
region = 'ap-south-1'  

try:
    s3.create_bucket(Bucket=bucket_name,
                     CreateBucketConfiguration={
                         'LocationConstraint': region
                     })
    print('S3 bucket has been created successfully ')
except Exception as e:
    print("S3 error: ", e)


S3 bucket has been created successfully 


In [10]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.0 MB/s eta 0:00:00:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 113.6 MB/s eta 0:00:0000:01
  Created wheel for peewee: filename=peewee-3.17.6-cp310-cp310-linux_x86_64.whl size=279106 sha256=82ecbb6595163d6025e6a68689f4a3c30cb52a8f5c6d9366fc2348c366e8709d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/4b/b9/b0/83d6e258e8f963f5ff111a2cd8c483ca59372a86e6a2535212
Successfully built peewee


In [11]:
import pandas as pd
from datetime import datetime
import yfinance as yf

In [12]:
start_date = datetime(2019, 1, 1)
end_date = datetime(2021, 1, 1)

df = yf.download('AAPL', start=start_date, end=end_date)

df.reset_index(inplace=True)

df

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,38.722500,39.712502,38.557499,39.480000,37.750095,148158800
1,2019-01-03,35.994999,36.430000,35.500000,35.547501,33.989899,365248800
2,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.440903,234428400
3,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.362015,219111200
4,2019-01-08,37.389999,37.955002,37.130001,37.687500,36.036125,164101200
...,...,...,...,...,...,...,...
500,2020-12-24,131.320007,133.460007,131.100006,131.970001,129.189529,54930100
501,2020-12-28,133.990005,137.339996,133.509995,136.690002,133.810059,124486200
502,2020-12-29,138.050003,138.789993,134.339996,134.869995,132.028397,121047300
503,2020-12-30,135.580002,135.990005,133.399994,133.720001,130.902634,96452100


In [13]:
df.shape

(505, 7)

# ELT

In [14]:
df.drop(axis=1, columns=['Adj Close'], inplace=True)
df.drop(axis=1, columns=['Date'], inplace=True)

In [15]:
df_features = df.iloc[:-1, :]
df_features

,Open,High,Low,Close,Volume
0,38.722500,39.712502,38.557499,39.480000,148158800
1,35.994999,36.430000,35.500000,35.547501,365248800
2,36.132500,37.137501,35.950001,37.064999,234428400
3,37.174999,37.207500,36.474998,36.982498,219111200
4,37.389999,37.955002,37.130001,37.687500,164101200
...,...,...,...,...,...
499,132.160004,132.429993,130.779999,130.960007,88223700
500,131.320007,133.460007,131.100006,131.970001,54930100
501,133.990005,137.339996,133.509995,136.690002,124486200
502,138.050003,138.789993,134.339996,134.869995,121047300


In [16]:
df_targets = df.iloc[1:, 0].rename('Targets')
df_targets

1       35.994999
2       36.132500
3       37.174999
4       37.389999
5       37.822498
          ...    
500    131.320007
501    133.990005
502    138.050003
503    135.580002
504    134.080002
Name: Targets, Length: 504, dtype: float64

In [20]:
df_features['Target'] = list(df_targets)

first_col = df_features.pop('Target')
df_features.insert(0, 'Target', first_col)

df_final = df_features
df_final.drop('Targt', axis=1, inplace=True)
df_final

/tmp/ipykernel_8109/3402416393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['Target'] = list(df_targets)
/tmp/ipykernel_8109/3402416393.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.drop('Targt', axis=1, inplace=True)


,Target,Open,High,Low,Close,Volume
0,35.994999,38.722500,39.712502,38.557499,39.480000,148158800
1,36.132500,35.994999,36.430000,35.500000,35.547501,365248800
2,37.174999,36.132500,37.137501,35.950001,37.064999,234428400
3,37.389999,37.174999,37.207500,36.474998,36.982498,219111200
4,37.822498,37.389999,37.955002,37.130001,37.687500,164101200
...,...,...,...,...,...,...
499,131.320007,132.160004,132.429993,130.779999,130.960007,88223700
500,133.990005,131.320007,133.460007,131.100006,131.970001,54930100
501,138.050003,133.990005,137.339996,133.509995,136.690002,124486200
502,135.580002,138.050003,138.789993,134.339996,134.869995,121047300


# Train_Test Split

In [21]:
import numpy as np
df_randomized = df_final.sample(frac=1, random_state=123)
df_randomized

,Target,Open,High,Low,Close,Volume
429,115.230003,118.330002,118.830002,113.610001,115.540001,184642000
282,78.839996,81.184998,81.495003,80.712502,81.237503,80113600
383,97.264999,95.334999,95.980003,94.705002,95.919998,90257200
477,117.180000,118.639999,118.769997,117.290001,117.339996,73604300
428,118.330002,114.720001,115.930000,112.800003,115.360001,140150100
...,...,...,...,...,...,...
98,45.049999,44.950001,45.134998,44.452499,44.915001,146118800
476,118.639999,117.589996,119.059998,116.809998,118.639999,74113000
322,70.599998,70.000000,72.062500,69.512497,71.762497,194994800
382,95.334999,96.262497,96.317497,94.672501,95.752502,125642800


In [22]:
train_data, test_data = np.split(df_randomized, [int(0.8*len(df_randomized))])
print(train_data.shape, test_data.shape)

(403, 6) (101, 6)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


# S3 Path Setup and Data Upload

In [23]:
import os

prefix = 'xgboost-as-a-built-n-algo'

train_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'train', 'train.csv')
test_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'test', 'test.csv')

print(train_csv_path)
print(test_csv_path)

s3://stockpricepredictionxgb/xgboost-as-a-built-n-algo/train/train.csv
s3://stockpricepredictionxgb/xgboost-as-a-built-n-algo/test/test.csv


In [24]:
train_data.to_csv(train_csv_path, index=False, header=False)
test_data.to_csv(test_csv_path, index=False, header=False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


# Build XGB Model

In [26]:
import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


# Find XGBoost Image and build XGBoost container

In [30]:
xgboost_container = image_uris.retrieve('xgboost', boto3.Session().region_name, '1.2-2')
display(xgboost_container)

'720646828776.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-xgboost:1.2-2'

# Initialize Hyperparameters

In [31]:
hyperparameters = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"reg:squarederror",
    "early_stopping_rounds":10,
    "num_round":1000
}

# output_Path

In [32]:
output_path = 's3://{}/{}/{}'.format(bucket_name, prefix, 'output')

print(output_path)

s3://stockpricepredictionxgb/xgboost-as-a-built-n-algo/output


# construct sagemaker estimator that calls the xgb container

In [33]:
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container,
                                         hyperparameters=hyperparameters,
                                         role = sagemaker.get_execution_role(),
                                         instance_count=1,
                                         instance_type='ml.m4.xlarge',
                                         volume_size=5,
                                         output_path=output_path,
                                         use_spot_instances=True,
                                         max_run=300,
                                         max_wait=600
                                         )

In [34]:
content_type = 'csv'
train_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name,prefix,'train'),content_type=content_type)
test_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name,prefix,'test'),content_type=content_type)

# Execute xgboost training job

In [35]:
estimator.fit({'train':train_input, 'validation':test_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-09-25-07-20-07-556


2024-09-25 07:20:10 Starting - Starting the training job...
2024-09-25 07:20:33 Starting - Preparing the instances for training......
2024-09-25 07:21:34 Downloading - Downloading the training image.........
2024-09-25 07:22:55 Training - Training image download completed. Training in progress..[2024-09-25 07:23:07.669 ip-10-0-152-144.ap-south-1.compute.internal:8 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-09-25:07:23:07:INFO] Imported framework sagemaker_xgboost_container.training
[2024-09-25:07:23:07:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2024-09-25:07:23:07:INFO] No GPUs detected (normal if no gpus installed)
[2024-09-25:07:23:07:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-09-25:07:23:07:INFO] Determined delimiter of CSV input is ','
[2024-09-25:07:23:07:INFO] Determined delimiter of CSV input is ','
[2024-09-25:07:23:07:INFO] Determined delimiter of CSV input is ','
[2024-09-25:07:23:07

# API Endpoint

In [36]:
from sagemaker.serializers import CSVSerializer
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge',serializer=CSVSerializer())

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-09-25-07-58-01-343
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-09-25-07-58-01-343
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-09-25-07-58-01-343


------!

In [37]:
xgb_predictor.endpoint_name

'sagemaker-xgboost-2024-09-25-07-58-01-343'

# Make Predictions using Endpoints

In [38]:
start_date = datetime(2021, 1, 4)
end_date = datetime(2021, 1, 5)

df = yf.download('AAPL', start = start_date, end=end_date)
df.reset_index(inplace=True)
df

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,133.520004,133.610001,126.760002,129.410004,126.683456,143301900


In [39]:
df.drop(axis=1,columns=['Adj Close'],inplace=True)
df.drop(axis=1,columns=['Date'],inplace=True)

data_features_array = df.values
data_features_array

array([[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08]])

# Inference - serialized input by sagemaker func

In [40]:
y_pred_fcn = xgb_predictor.predict(data_features_array).decode('utf-8')
print(y_pred_fcn)

132.41209411621094



In [41]:
from sagemaker.serializers import CSVSerializer

serialized_input_fcn = CSVSerializer().serialize([[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08]])
print(serialized_input_fcn)


133.520004,133.610001,126.760002,129.410004,143301900.0


In [42]:
y_pred_fcn

'132.41209411621094\n'

# Inference - serialized input by built-in-func (Lambda func friendly)

In [48]:
Input = [[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08],
        [1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08],
        [1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08]]
        
Serialized_Input = ','.join(map(str,Input[0]))
print(Serialized_Input)

133.520004,133.610001,126.760002,129.410004,143301900.0


In [49]:
y_pred = xgb_predictor.predict(Serialized_Input).decode('utf-8')
y_pred

'132.41209411621094\n'

In [51]:
import boto3
ENDPOINT_NAME = 'sagemaker-xgboost-2024-09-25-07-58-01-343'

runtime = boto3.client('runtime.sagemaker')

def lambda_handler(event,context):
    inputs = event['data']
    result = []
    for input in inputs:
        serialized_input = ','.join(map(str, input))
        
        response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                          ContentType='text/csv',
                                          Body=serialized_input)
        result.append(response['Body'].read().decode().strip())
    return result
input_json = {'data':
             [[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08],
        [1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08],
        [1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08]]}
result = lambda_handler(input_json,__)
result

['132.41209411621094', '132.41209411621094', '132.41209411621094']

# Testing API 

In [53]:
import requests
API_ENDPOINT = "https://6dpsfoo4u9.execute-api.ap-south-1.amazonaws.com/xgbmodel"

json = {'data':
             [[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08],
        [1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08],
        [1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08]]}

r = requests.post(url=API_ENDPOINT, json=json)

In [54]:
print(f"Status Code: {r.status_code}, Response: {r.json()}")

Status Code: 200, Response: ['132.41209411621094', '132.41209411621094', '132.41209411621094']


# Close and Terminate

In [ ]:
sagemaker.session().delete_endpoint(xgb_predictor.endpoint)

In [ ]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()